In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [56]:
class ToFCalculator():
    """ ToF calculator for a single defect and single scan position. 
    """
    def __init__(self):
        self.c0 = None
        self.tof = None
        self.grad_tof = None
        
    def get_angle(self, p_def, p_scan):
        origin = p_scan
        vec1 = np.array([0.0, 1.0])
        vec2 = p_def - origin
        # angle b/w two vectors
        num = np.dot(vec1, vec2)
        den = np.linalg.norm(vec2)
        if den == 0:
            angle = 0
        else:
            angle = np.rad2deg(np.arccos(num/den))
        return angle
        
    
    def get_tau(self, p_def, p_scan):
        return 2*np.linalg.norm(p_def - p_scan)/ self.c0 #[S], unitless
    
    def get_grad_tau(self, x_def, x_scan, tau):
        if tau == 0.0:
            return 0.0
        else:
            return 4* (x_def - x_scan)/ (tau* self.c0**2) # multiply by "-"??? -> check!! (19.12.21)
    
    def get_tof(self):
        return self.tof
    
    def get_grad_tof(self):
        return self.grad_tof

    
class ToFforDictionary2D(ToFCalculator):
    """ ToF calculator for a dictionary formation. 
    """
    def __init__(self, c0, Nx, Nz, dx, dz, p_scan_all):
        self.c0 = float(c0)
        self.Nx = int(Nx)
        self.Nz = int(Nz)
        self.dx = float(dx)
        self.dz = float(dz)
        if p_scan_all.ndim == 1:
            self.p_scan_all = np.array([p_scan_all]) # otherwise x and z are handled separately
        else:
            self.p_scan_all = np.array(p_scan_all)
        
    def calculate_tof(self, calc_grad = False, opening_angle = 180):
        """
        Parameter
        ---------
        calc_grad: boolean
            True, if gradient (derivative) of tau should be calculated
        opening_angle: int, float in [deg]
            To restrict the region to consider, default value is 180 [deg]
            
        Test
        ----
        0.5* tau* c0 = np.linalg.norm(p_def - p_scan)
        """
        # Base of tof and grad_tof
        self.tof = np.zeros((self.Nx* self.Nz, self.p_scan_all.shape[0]))
        if calc_grad == True:
            self.grad_tof = np.array(self.tof)
            
        for k, p_scan in enumerate(self.p_scan_all):
            for x_idx in range(self.Nx):
                for z_idx in range(self.Nz):
                    l = x_idx* self.Nz + z_idx
                    p_def = np.array([x_idx*self.dx, z_idx*self.dz])
                    # Constraint: opening angle
                    # -> Consider the region only within the opening angle
                    angle = self.get_angle(p_def, p_scan)
                    if angle <= 0.5* opening_angle:
                        tau = self.get_tau(p_def, p_scan)
                    else:
                        tau = 0
                    self.tof[l, k] = tau
                    if calc_grad == True:
                        self.grad_tof[l, k] = self.get_grad_tau(p_def[0], p_scan[0], tau)

In [48]:
Nx = 10 # limited due to the opening angle
Nz = 200
Nt = Nz
c0 = 6300 #[m/S]
fS = 80*10**6 #[Hz] 
fC = 5*10**6 #[Hz] 
alpha = 20*10**12 #[Hz]**2
dx = 0.5*10**-3 #[m]
dz = 0.5* c0/(fS)
wavelength = 1.26* 10**-3 # [m]
# defect position: p_defect
p_def_idx = np.array([10, 91])
p_def = np.array([p_def_idx[0]*dx, p_def_idx[1]*dz])

In [29]:
# Reflecting the OA to the rof calculator
p_scan = np.array([[0*dx, 0*dz], [5*dx, 0*dz]])
origin = p_scan[0]
vec1 = np.array([0.0, 1.0])
vec2 = p_def - origin
# angle b/w two vectors
num = np.dot(vec1, vec2)
den = np.linalg.norm(vec2)
angle = np.rad2deg(np.arccos(num/den))
print(angle)
print(origin)

12.537914135413951
[0. 0.]


In [57]:
tofcalc = ToFforDictionary2D(c0, Nx, Nz, dx, dz, np.array([[4*dx, 0*dz], [3*dx, 0*dz]]))
tofcalc.calculate_tof(calc_grad = True)
tof = tofcalc.get_tof()
grad_tof = tofcalc.get_grad_tof()

In [58]:
tof[0]

array([6.34920635e-07, 4.76190476e-07])

In [59]:
opening_angle = 25 #[degree]
tofcalc = ToFforDictionary2D(c0, Nx, Nz, dx, dz, np.array([[4*dx, 0*dz], [3*dx, 0*dz]]))
tofcalc.calculate_tof(calc_grad = True, opening_angle = 25)
tof = tofcalc.get_tof()
grad_tof = tofcalc.get_grad_tof()

In [60]:
tof[0]

array([0., 0.])